In [ ]:
%matplotlib inline
import numpy as np
from skyfield.api import load
from scipy.optimize import newton
from matplotlib import pyplot as plt
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
from footprint import *
from tno_models import trojan, plutino, twotino
init_notebook_mode(connected=True)
planets = load('de423.bsp')
plotly.tools.set_credentials_file(username='sevenlin123', api_key='vhjhhsCHepMx5oQGJPEp')

[#################                ]  52% de422.bsp

In [ ]:
#p = plutino(size = 2000, mjd=58199, e_c = 0.3, e_sigma = 0.01, amp_c = 1, amp_max = 2, amp_min = 0)
p = plutino(size = 5000, mjd=57023)

In [ ]:
plt.hist(p.a, bins = 20)

In [ ]:
plt.hist(p.e, bins = 20)

In [ ]:
plt.hist(p.i*180/np.pi, bins = 20)

In [ ]:
plt.yscale('log', nonposy='clip')
plt.hist(p.H, bins = 20)

In [ ]:
plt.yscale('log', nonposy='clip')
plt.hist(p.mag_r, bins = 20)

In [ ]:
plt.hist(p.M, bins = 20)

In [ ]:
plt.hist(p.arg, bins = 20)

In [ ]:
plt.hist(p.node, bins = 20)

In [ ]:
plt.hist((p.arg+p.node) % (2*np.pi), bins = 20)

In [ ]:
plt.hist(p.phi, bins = 20)

In [ ]:
def kep_to_xyz(a, e, i, arg, node, M):
    # compute eccentric anomaly
    f = lambda E, M, e: E - e * np.sin(E) - M
    E0 = M
    E = newton(f, E0, args=(M, e))
    # compute true anomaly
    v = 2 * np.arctan2((1 + e)**0.5*np.sin(E/2.), (1 - e)**0.5*np.cos(E/2.))
    # compute the radius
    r = a * (1 - e*np.cos(E))
    # compute X,Y,Z
    X = r * (np.cos(node) * np.cos(arg + v) - np.sin(node) * np.sin(arg + v) * np.cos(i))
    Y = r * (np.sin(node) * np.cos(arg + v) + np.cos(node) * np.sin(arg + v) * np.cos(i))
    Z = r * (np.sin(i) * np.sin(arg + v))
    return X, Y, Z

In [ ]:
xn, yn, zn = zip(*map(kep_to_xyz, 30+np.zeros(1000), np.zeros(1000), np.zeros(1000), np.zeros(1000), np.zeros(1000), np.arange(0, 6.28, 0.00628)))
x45, y45, z45 = zip(*map(kep_to_xyz, 45+np.zeros(1000), np.zeros(1000), np.zeros(1000), np.zeros(1000), np.zeros(1000), np.arange(0, 6.28, 0.00628)))

In [ ]:
plt.axis('equal')
plt.scatter(p.X,p.Y, marker = '.')
plt.plot(xn, yn, 'y')
plt.plot(x45, y45, 'y')
plt.plot(p.x_n, p.y_n, 'ro')

In [ ]:
plt.axis('equal')
plt.scatter(p.X,p.Z, marker = '.')

In [ ]:
def buildmap(obj_ra, obj_dec, ecliptic_plots=True):
    lon, lat = define_footprint(ecliptic_plots=ecliptic_plots) 
#    lon2, lat2 = define_footprint(polydef='poly_bliss_p9.txt', ecliptic_plots=ecliptic_plots) 
    m = Basemap(lon_0=0, projection='moll', celestial=True)
    x, y = m( lon, lat )
#    x2,y2 = m(lon2, lat2)
    xy = zip(x,y)
#    xy2 = zip(x2,y2)
    foot = patches.Polygon( xy, facecolor='cornflowerblue', edgecolor=None, alpha=0.4 )
#    foot2 = patches.Polygon( xy2, facecolor='lightpink', edgecolor=None, alpha=0.4 )
    plt.gca().add_patch(foot)
#    plt.gca().add_patch(foot2)
    fields = SNfields()
    for f in fields:
        if ecliptic_plots:
            ecl = ephem.Ecliptic(ephem.Equatorial(f.a_ra, f.a_dec))
            lon = ecl.lon if ecl.lon<ephem.degrees('180') else ecl.lon-2*np.pi    
            m.tissot(lon*180/np.pi, ecl.lat*180/np.pi, 1.05, 100, facecolor='g', alpha=0.5)
        else:
            ra = f.a_ra if f.a_ra<ephem.degrees('180') else f.a_ra-2*np.pi
            m.tissot(ra*180/np.pi, f.a_dec*180/np.pi, 1.05, 100, facecolor='r', alpha=0.5)
    obj_ra = obj_ra*180/np.pi
    obj_dec = obj_dec*180/np.pi
    for i in range(len(obj_ra)):
        m.scatter(obj_ra[i], obj_dec[i],3,marker='.',color='k', latlon=True)
    m.drawmapboundary()
    parallels = np.arange(-180.,181,20.)
    m.drawparallels(parallels,labels=[False,True,True,False], alpha=0.4)
    meridians = np.arange(-180.,181.,20.)
    m.drawmeridians(meridians, alpha=0.4)
    return m

In [ ]:
plt.figure(figsize=(20,10))
m = buildmap(p.ra, p.dec, ecliptic_plots=False)

In [ ]:
plutinos = Scatter3d(x=p.X, y=p.Y, z=p.Z, mode='markers', marker=dict(size=1, symbol='circle'), opacity=0.7, name='plutinos')
sun = Scatter3d(x=0, y=0, z=0, mode='markers', marker=dict(size=10, symbol='circle'), opacity=1, name='Sun')
neptune = Scatter3d(x=p.x_n, y=p.y_n, z=p.z_n, mode='markers', marker=dict(size=5, symbol='circle'), opacity=0.7, name ='Neptune')
fig = Figure(data=[plutinos, sun, neptune])
py.iplot(fig)

In [ ]:
t = trojan(size = 5000, mjd=57023)

In [ ]:
plt.hist(t.a, bins = 20)

In [ ]:
plt.hist(t.e, bins = 20)

In [ ]:
plt.hist(t.i*180/np.pi, bins = 20)

In [ ]:
plt.hist(t.amp*180/np.pi, bins = 20)

In [ ]:
plt.hist((t.phi+t.lambda_N)%(2*np.pi)*180/np.pi, bins = 20)

In [ ]:
plt.yscale('log')
plt.hist(t.mag_r, bins = 20)

In [ ]:
plt.axis('equal')
plt.scatter(t.X,t.Y, marker = '.')
plt.plot(xn, yn, 'y')
plt.plot(x45, y45, 'y')
plt.plot(t.x_n, t.y_n, 'ro')

In [ ]:
trojan = Scatter3d(x=t.X, y=t.Y, z=t.Z, mode='markers', marker=dict(size=1, symbol='circle'), opacity=0.7, name='plutinos')
sun = Scatter3d(x=0, y=0, z=0, mode='markers', marker=dict(size=10, symbol='circle'), opacity=1, name='Sun')
neptune = Scatter3d(x=t.x_n, y=t.y_n, z=t.z_n, mode='markers', marker=dict(size=5, symbol='circle'), opacity=0.7, name ='Neptune')
fig = Figure(data=[trojan, sun, neptune])
py.iplot(fig)

In [ ]:
plt.figure(figsize=(20,10))
m = buildmap(t.ra, t.dec, ecliptic_plots=False)

In [ ]:
two = twotino(size = 5000, mjd=57023, e_c = 0.4, amp_c = 3, amp_max = 5, amp_min = 0)

In [ ]:
plt.hist((two.i)*180/np.pi, bins = 20)

In [ ]:
plt.hist((two.phi+two.lambda_N)%(2*np.pi)*180/np.pi, bins = 20)

In [ ]:
plt.yscale('log')
plt.hist(t.mag_r, bins = 20)

In [ ]:
plt.axis('equal')
plt.scatter(two.X[two.phi0 == 0],two.Y[two.phi0 == 0], marker = '.')
plt.plot(xn, yn, 'y')
plt.plot(x45, y45, 'y')
plt.plot(two.x_n, two.y_n, 'ro')

In [ ]:
plt.axis('equal')
plt.scatter(two.X[two.phi0 != 0],two.Y[two.phi0 != 0], marker = '.')
plt.plot(xn, yn, 'y')
plt.plot(x45, y45, 'y')
plt.plot(two.x_n, two.y_n, 'ro')

In [ ]:
plt.axis('equal')
plt.scatter(two.X,two.Y, marker = '.')
plt.plot(xn, yn, 'y')
plt.plot(x45, y45, 'y')
plt.plot(two.x_n, two.y_n, 'ro')

In [ ]:
twotinos = Scatter3d(x=two.X, y=two.Y, z=two.Z, mode='markers', marker=dict(size=1, symbol='circle'), opacity=0.7, name='twotinos')
sun = Scatter3d(x=0, y=0, z=0, mode='markers', marker=dict(size=10, symbol='circle'), opacity=1, name='Sun')
neptune = Scatter3d(x=two.x_n, y=two.y_n, z=two.z_n, mode='markers', marker=dict(size=5, symbol='circle'), opacity=0.7, name ='Neptune')
fig = Figure(data=[twotinos, sun, neptune])
py.iplot(fig)

In [ ]:
plt.figure(figsize=(20,10))
m = buildmap(two.ra[two.phi0 == 0], two.dec[two.phi0 == 0], ecliptic_plots=False)

In [ ]:
plt.figure(figsize=(20,10))
m = buildmap(two.ra[two.phi0 != 0], two.dec[two.phi0 != 0], ecliptic_plots=False)